### Heroes Of Pymoli Data Analysis
* While females and other/non-disclosed make up significantly less of the population of players than males, the two groups spend more on average and purchase higher priced items.  This may converge if sales pickup, but definitely something to at least dig into more to determine if more resources should be allocated allocated to grow the female segment.

* The peak age demographic of 20-24 (44.8%) performs relatively well on average spent per customer as well as average price of item purchased even though more total sales (see opposite trend when it comes to gender for dominant male group).  There is also a significant bump up in each of those metrics in the 30's age groups.

* There appears to be a positive relationship between the popularity of an item and its profitability.  Given this relationship along with the above two observations, it is probably worth further investigation whether future resources should be more heavily allocated to promoting these higher priced/higher profitability items to a potentially under reached female demographic and 30 year old demographic.  The peak age demographic should continue to be a focus as well given the seemingly similar strong performance with the higher profitability items.
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [216]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [217]:
#get unique players based on SN
player_list = purchase_data["SN"].unique()

#count unique players and put into data frame
player_count = pd.DataFrame({"total players": len(player_list)}, index=[0])

player_count

,total players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [218]:
#run total calculations on availabe data
unique_items = purchase_data["Item ID"].unique()
average_price = purchase_data["Price"].mean()
number_of_purchases = len(purchase_data["Purchase ID"].unique())
total_revenue = number_of_purchases * average_price

#make data frame to hold result of calculation
purchasing_analysis_total = pd.DataFrame({"number of unique items":len(unique_items), "average price":average_price, "number of purchases":number_of_purchases, "total revenue":total_revenue}, index=[0])

#format data frame above (locked and not a new formatted one created)
purchasing_analysis_total["average price"] = purchasing_analysis_total["average price"].map("${:.2f}".format)
purchasing_analysis_total["total revenue"] = purchasing_analysis_total["total revenue"].map("${:,.2f}".format)

purchasing_analysis_total

,number of unique items,average price,number of purchases,total revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [219]:
#create data frame from original with just SN and gender
gender_df = purchase_data[["SN","Gender"]]

#drop duplicates based on SN to leave unique players
gender_group_df = gender_df.drop_duplicates()

#count unique players and send to dictionary with gender as key
gender_demo_dic = gender_group_df["Gender"].value_counts().to_dict()

#take gender dictionary back into data frame with renamed total count column
gender_demo_df = pd.DataFrame.from_dict(gender_demo_dic, orient="index", columns=["total count"])

#calculate percentage of total for each gender with player_list previously generated
percentage_players_gender = round(gender_demo_df["total count"]/len(player_list)*100, 2)

#add percentage calc into data frame
gender_demo_df["percentage of players"] = percentage_players_gender

gender_demo_df

,total count,percentage of players
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [220]:
#take original data frame and groupy gender and aggregate price column by count and sum
purchasing_analysis_gender = purchase_data.groupby("Gender")["Price"].agg(["count","sum"])

#calculate average purchase price
avg_purchase_price = round(purchasing_analysis_gender["sum"]/purchasing_analysis_gender["count"],2)

#calculate average purchase price per person
avg_purchase_price_person = round(purchasing_analysis_gender["sum"]/gender_demo_df["total count"],2)

#add calculations to data frame
purchasing_analysis_gender["average purchase price"] = avg_purchase_price
purchasing_analysis_gender["avg total purchase per person"] = avg_purchase_price_person

#rename aggregated columns from groupby
purchasing_analysis_gender = purchasing_analysis_gender.rename(columns={"count":"purchase count", "sum":"total purchase value"})

#create new data frame with formatting so original dataframe can be referenced later in calc if necessary
purchasing_analysis_gender_format = purchasing_analysis_gender[["purchase count","average purchase price","total purchase value","avg total purchase per person"]]
purchasing_analysis_gender_format["average purchase price"] = purchasing_analysis_gender_format["average purchase price"].map("${:,.2f}".format)
purchasing_analysis_gender_format["total purchase value"] = purchasing_analysis_gender_format["total purchase value"].map("${:,.2f}".format)
purchasing_analysis_gender_format["avg total purchase per person"] = purchasing_analysis_gender_format["avg total purchase per person"].map("${:,.2f}".format)

purchasing_analysis_gender_format

,purchase count,average purchase price,total purchase value,avg total purchase per person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [221]:
#print min and max age from original data frame to help determine bins
print(purchase_data["Age"].max())
print(purchase_data["Age"].min())

45
7


In [222]:
#create bins for ages
bins = [0, 9, 14, 19, 24, 29, 34, 39, 45]

#create labels for bins
group_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#add new column with group labels based on bins to original data frame
purchase_data["Age Group"] = pd.cut(purchase_data["Age"], bins, labels = group_labels)

#create data frame from original with just SN and age bins
age_df = purchase_data[["SN","Age Group"]]

#drop duplicates based on SN to leave unique players
age_group_df = age_df.drop_duplicates()

#count unique players and send to dictionary with age bins as key
age_demo_dic = age_group_df["Age Group"].value_counts().to_dict()

#calculate percentage of total for each age bin with player_list previously generated and reorder rows based on bin labels
age_demo_df = pd.DataFrame.from_dict(age_demo_dic, orient="index", columns=["total count"]).reindex(group_labels)

#calculate percentage of total for each age bin with player_list previously generated
percentage_players_age = round(age_demo_df["total count"]/len(player_list)*100, 2)

#add percentage calc into data frame
age_demo_df["percentage of players"] = percentage_players_age

age_demo_df

,total count,percentage of players
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [223]:
#take original data frame and groupy age bins and aggregate price column by count and sum
purchasing_analysis_age = purchase_data.groupby("Age Group")["Price"].agg(["count","sum"])

#calculate average purchase price
avg_purchase_price_age = round(purchasing_analysis_age["sum"]/purchasing_analysis_age["count"],2)

#calculate average purchase price per person
avg_purchase_price_person_age = round(purchasing_analysis_age["sum"]/age_demo_df["total count"],2)

#add calculations to data frame
purchasing_analysis_age["average purchase price"] = avg_purchase_price_age
purchasing_analysis_age["avg total purchase per person"] = avg_purchase_price_person_age

#rename aggregated columns from groupby
purchasing_analysis_age = purchasing_analysis_age.rename(columns={"count":"purchase count", "sum":"total purchase value"})

#create new data frame with formatting so original dataframe can be referenced later in calc if necessary
purchasing_analysis_age_format = purchasing_analysis_age[["purchase count","average purchase price","total purchase value","avg total purchase per person"]]
purchasing_analysis_age_format["average purchase price"] = purchasing_analysis_age_format["average purchase price"].map("${:,.2f}".format)
purchasing_analysis_age_format["total purchase value"] = purchasing_analysis_age_format["total purchase value"].map("${:,.2f}".format)
purchasing_analysis_age_format["avg total purchase per person"] = purchasing_analysis_age_format["avg total purchase per person"].map("${:,.2f}".format)

purchasing_analysis_age_format

,purchase count,average purchase price,total purchase value,avg total purchase per person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [224]:
#take original data frame and groupy SN and aggregate price column by count and sum
purchasing_analysis_spenders = purchase_data.groupby("SN")["Price"].agg(["count","sum"])

#calculate average purchase price
avg_purchase_price_spenders = round(purchasing_analysis_spenders["sum"]/purchasing_analysis_spenders["count"],2)

#add calculations to data frame
purchasing_analysis_spenders["average purchase price"] = avg_purchase_price_spenders

#rename aggregated columns from groupby
purchasing_analysis_spenders = purchasing_analysis_spenders.rename(columns={"count":"purchase count", "sum":"total purchase value"})

#sort data fram in descending order based on total purchase value
purchasing_analysis_spenders = purchasing_analysis_spenders.sort_values("total purchase value", ascending=False)

#create new data frame with formatting so original dataframe can be referenced later in calc if necessary
purchasing_analysis_spenders_format = purchasing_analysis_spenders[["purchase count","average purchase price","total purchase value"]]
purchasing_analysis_spenders_format["average purchase price"] = purchasing_analysis_spenders_format["average purchase price"].map("${:,.2f}".format)
purchasing_analysis_spenders_format["total purchase value"] = purchasing_analysis_spenders_format["total purchase value"].map("${:,.2f}".format)

purchasing_analysis_spenders_format.head()

,purchase count,average purchase price,total purchase value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [225]:
#take original data frame and groupy SN and aggregate price column by count and sum
purchasing_analysis_popular = purchase_data.groupby(["Item ID","Item Name"])["Price"].agg(["count","sum"])

#calculate item price
item_price = round(purchasing_analysis_popular["sum"]/purchasing_analysis_popular["count"],2)

#add calculations to data frame
purchasing_analysis_popular["item price"] = item_price

#rename aggregated columns from groupby
purchasing_analysis_popular = purchasing_analysis_popular.rename(columns={"count":"purchase count", "sum":"total purchase value"})

#sort data fram in descending order based on purchase count
purchasing_analysis_popular = purchasing_analysis_popular.sort_values("purchase count", ascending=False)

#create new data frame with formatting so original dataframe can be referenced later in calc if necessary
purchasing_analysis_popular_format = purchasing_analysis_popular[["purchase count","item price","total purchase value"]]
purchasing_analysis_popular_format["item price"] = purchasing_analysis_popular_format["item price"].map("${:,.2f}".format)
purchasing_analysis_popular_format["total purchase value"] = purchasing_analysis_popular_format["total purchase value"].map("${:,.2f}".format)

purchasing_analysis_popular_format.head()

,,purchase count,item price,total purchase value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [226]:
#resort non-formated popular data frame in descending order based on total purchase value
purchasing_analysis_profit = purchasing_analysis_popular.sort_values("total purchase value", ascending=False)

#create new data frame with formatting so original dataframe can be referenced later in calc if necessary
purchasing_analysis_profit_format = purchasing_analysis_profit[["purchase count","item price","total purchase value"]]
purchasing_analysis_profit_format["item price"] = purchasing_analysis_profit_format["item price"].map("${:,.2f}".format)
purchasing_analysis_profit_format["total purchase value"] = purchasing_analysis_profit_format["total purchase value"].map("${:,.2f}".format)

purchasing_analysis_profit_format.head()

,,purchase count,item price,total purchase value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
